## Getting started and authenticating
 Run this code to authenticate.Gs_quant requires to be initialized session with your credentials. Provide 'client_id' and 'client_secret' to session initializer. If you see an error, please e-mail the error message to gs-marquee-help@gs.com..

In [ ]:
import datetime
import json

from gs_quant.target.secmaster import SecMasterAssetType, SecMasterIdentifiers
from gs_quant.api.gs.secmaster import GsSecurityMasterApi
from gs_quant.markets.securities import SecurityIdentifier
from gs_quant.session import GsSession
import pandas as pd
GsSession.use(client_id="",
              client_secret="")

### Pagination
Whenever you encounter an `offsetKey` in the response it means that results did not fit the page size, and this requires a more requests to get all data.

Initial call that  returns first `offsetKey`

In [ ]:
results  = GsSecurityMasterApi.get_many_securities(type_=SecMasterAssetType.Common_Stock, limit=500)
offset_key = results['offsetKey']
print(f"Offset key is {offset_key}")
pd.DataFrame(results['results'][-5:])[["id", "name", "currency", "assetClass", "type", "effectiveDate"]]

Subsequent call using last `offsetKey` for next call. It returns next page of  data. This pattern can be applied to all instances where `offsetKey` is returned

In [ ]:
results  = GsSecurityMasterApi.get_many_securities(type_=SecMasterAssetType.Common_Stock, limit=10, offset_key=offset_key)
offset_key = results['offsetKey']
print(f"New offset key is {offset_key}")
pd.DataFrame(results['results'][-5:])[["id", "name", "currency", "assetClass", "type", "effectiveDate"]]

## Securities
The Securities section is the analytical core of the service, focusing on providing asset-specific data. It is designed for users who require detailed information about individual securities, need to trace the history and changes of security identifiers, or wish to obtain bulk security data by asset type.

#### Historical query

The historical query functionality is tailored for users requiring time-specific data retrieval. It allows you to perform a point-in-time query by specifying an identifier (e.g. bbid) and a precise date. This can be important for tasks such as data verification, back-testing models against historical data.
It effectively enables you to 'query back in time', providing a snapshot of security's data as it existed at that exact moment

In [ ]:
data = GsSecurityMasterApi.get_security_data(id_value="META UW", id_type=SecurityIdentifier.BBID,
                                             effective_date=datetime.date(2024, 10, 28))
print( f"The bbid is: {data['identifiers.bbid']} and the issuer name is: {data['issuer.name']}.")  # Prints bbid and company name

#### Query by identifier


This functionality allows you to immediate access to current data on specific security. By inputting an identifier, users can quickly retrieve the latest available information about the security

In [ ]:

data = GsSecurityMasterApi.get_security_data(id_value="GS UN", id_type=SecurityIdentifier.BBID)
print( f"The bbid is: {data['identifiers.bbid']} and the issuer name is: {data['issuer.name']}.")

#### Query by attribute

This functionality is a versatile tool for customized queries. Users can specify an asset type, or set of identifiers to retrieve matching securities. This function accepts multiple identifiers, either individually or in batch, enabling a broad or targeted browse.
It is particular useful for users how need to extract a specific subset of data from a vast pool of securities, aligning with particular investment strategies, or research parameters.


In [ ]:
etf  = GsSecurityMasterApi.get_many_securities(type_=SecMasterAssetType.ETF, limit=5)
[ print( f"The bbid is: {data['identifiers']['bbid']} and the issuer name is: {data['issuer']['name']}.") for data in etf["results"]]
print("-"*80)
single_asset = GsSecurityMasterApi.get_many_securities(bbid="IZB PW")
[ print( f"The bbid is: {data['identifiers']['bbid']} and the issuer name is: {data['issuer']['name']}.") for data in single_asset["results"]]
print("-"*80)
many_assets = GsSecurityMasterApi.get_many_securities(ric=["GOOGL.OQ", "VOD.L"])
[ print( f"The bbid is: {data['identifiers']['bbid']} and the issuer name is: {data['issuer']['name']}.") for data in many_assets["results"]]
print("-"*80)
ticker = GsSecurityMasterApi.get_many_securities(ticker="GS")
[ print( f"The name is: {data['name']} and the issuer name is: {data['issuer']['name']}.") for data in ticker["results"]]
print("-"*80)

#### Get all securities
This acts as an expansive layer over 'Query by Attribute', tailored for retrieving an extensive list of assets within a specific type, It simplifies the user experience by automatically handling the scrolling, thereby providing a seamless method to acquire a complete dataset of a chosen asset type.


In [ ]:
data = GsSecurityMasterApi.get_all_securities(type_=SecMasterAssetType.Dutch_Cert)
print(json.dumps(data['results'][0:60], indent=4))  # Prints prettified response


**NOTE** Not all functionalities are available for Base Tier.

#### Get identifier history
The 'Get identifier history' functionality is designed to provide a chronological account of how a particular identifier has changed for a security over time.

In [ ]:
secm_id = "GSPD227284E154"
data = GsSecurityMasterApi.get_identifiers(secmaster_id=secm_id)
pd.DataFrame(data)

#### Map securities

This functionality is a powerful tool for converting between different types of security identifiers while also taking historical changes int account. Users can input a security's identifier alon with its type (e.g, bbid, sedol) and specify  the desired output identifier types. This function then returns a mapping of the security across these different identifier systems.

In [ ]:
mapping = GsSecurityMasterApi.map(SecurityIdentifier.BBID, ["GS UN"], start_date="2015-01-01", end_date="2020-01-01",
                                  output_types=[SecurityIdentifier.TICKER, SecurityIdentifier.RIC])
pd.DataFrame(mapping)

#### Search securities

This functionality is a designed for conducting full-text searches across many fields, including identifiers, company names. This is especially beneficial for users who require a more exploratory approach to data retrieval.

This supports also filtering options, where users can refine their search results using filters such as 'isPrimary', 'activeListing' and restrict searches to selected asset type. This helps narrow down the results


##### Plain full text search query:

In [ ]:
results = GsSecurityMasterApi.search(q="TESLA")
[
    f"The bbid is: {data['identifiers']['bbid']} and the issuer name is: {data['issuer']['name']} and the type is: {data['type']}."
    for data in results
]

##### Full text query with filters

In [ ]:
results = GsSecurityMasterApi.search(q="TESLA", is_primary=True, active_listing=True, type_=SecMasterAssetType.Common_Stock)
[
    f"The bbid is: {data['identifiers']['bbid']} and the issuer name is: {data['issuer']['name']}  and the type is: {data['type']}."
    for data in results
]

#### Get recent changes
You can get the list of recent identifier changes by providing between start/end time. The start/end time are limited to span of 5 days and should not start more than 30 days from now
The API allows returning processed results highlighting the change nature (easier to interprate), or raw results easier to process programmatically

##### Raw results

In [ ]:
start_time = datetime.datetime.now() - datetime.timedelta(hours=12)
data = GsSecurityMasterApi.get_deltas(raw=True, start_time=start_time)
pd.DataFrame(data['results'])

##### Processed results

In [ ]:
start_time = datetime.datetime.now() - datetime.timedelta(hours=12)
data = GsSecurityMasterApi.get_deltas(raw=False, start_time=start_time)
sample = json.dumps(data['results'][:10], indent=4)
print(sample)

## Corporates

Secmaster service allows providing a detailed insights into financial structure and activities of listed companies.

### Capital Structure

This functionality is a tool for gaining a comprehensive understanding of different types of securities issued by a company. By utilizing various identifiers, users can pinpoint a specific company and retrieve a detailed list of its issued securities, including bonds, stocks and other.
It supports point-in-time query.

In [ ]:
from IPython.display import Markdown
ric = "VOD.L"
data = GsSecurityMasterApi.get_capital_structure(id_value=ric, id_type=SecurityIdentifier.RIC)
res = data['results'][0]
display(Markdown(f"Capital structure for {ric} matched {res['issuerName']}"))
display(pd.DataFrame([data['assetTypesTotal']]))
for t in res['types']:
    display(Markdown(f"**{t}**"))
    display(pd.DataFrame(res['types'][t][:5]))

### Corporate Actions

This functionality provides the ability to track both historical na upcoming corporate actions.

In [ ]:
corp = GsSecurityMasterApi.get_corporate_actions(id_value="227284")
pd.DataFrame(corp)[["corpActionId", "eventId", "eventType", "eventStatus", "gsid", "effectiveDate", "announceDate", "multiplicativeAdjust", "additiveAdjust"]]

## Exchanges

You can fetch data about exchanges. The API allows either fetching all exchanges or query by exchange identifier

###  Get  many exchanges
This functionally allows users to retrieve detailed information about exchanges. User can fetch all exchanges or query by selected identifiers.

In [ ]:
data = GsSecurityMasterApi.get_exchanges()
pd.DataFrame(data['results'][:5])

###  Query exchanges by identifiers

data = GsSecurityMasterApi.get_exchanges(mic="XBUE")
pd.DataFrame(data['results'])

In [ ]:
data = GsSecurityMasterApi.get_exchanges(mic="XBUE")
pd.DataFrame(data['results'])

### Get exchanges identifiers history

This functionality is designed to provide a historical view of the changes in exchange identifiers over time.

In [ ]:
results = GsSecurityMasterApi.get_exchange_identifiers_history(gs_exchange_id=1)
pd.DataFrame(results)